In [1]:
import os
os.chdir("/home/atulgang/Thesis")


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

2018-05-07 04:38:36,485 : INFO : 'pattern' package not found; tag filters are not available for English


In [4]:

# WordNet only cares about 5 parts of speech.
# The other parts of speech will be tagged as nouns.

part = {
    'N' : 'n',
    'V' : 'v',
    'J' : 'a',
    'S' : 's',
    'R' : 'r'
}

wnl = WordNetLemmatizer()

def convert_tag(penn_tag):
    '''
    convert_tag() accepts the **first letter** of a Penn part-of-speech tag,
    then uses a dict lookup to convert it to the appropriate WordNet tag.
    '''
    if penn_tag in part.keys():
        return part[penn_tag]
    else:
        # other parts of speech will be tagged as nouns
        return 'n'


def tag_and_lem(element):
    '''
    tag_and_lem() accepts a token list, tags, converts tags,
    lemmatizes, and returns a string
    '''
    # list of tuples [('token', 'tag'), ('token2', 'tag2')...]
    sent = pos_tag(element) # must tag in context
    return [ wnl.lemmatize(sent[k][0], convert_tag(sent[k][1][0])) for k in range(len(sent))]

In [5]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

In [6]:
en_stop = set(get_stop_words('en'))

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        #print filename
        # read the whole file as lowercase string
        string = (f.read()).lower()
        try:
            temp = word_tokenize(string)
        except:
            string = string.decode("latin-1")
            temp = word_tokenize(string)
        
        tokens = []
        # tokenize that string
        for word in temp:
            w =  only_alphabet(word).lower()
            if w not in en_stop:
                if w:
                    tokens.append(w)
        tokens = tag_and_lem(tokens)
        yield tokens
        f.close()

        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [7]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('dict/lem10')

2018-05-07 04:39:06,023 : INFO : loading Dictionary object from dict/lem10
2018-05-07 04:39:06,132 : INFO : loaded dict/lem10


In [8]:
######################################## load BOW
corpus = corpora.MmCorpus('corpus/bow_lem10.mm')

2018-05-07 04:39:06,208 : INFO : loaded corpus index from corpus/bow_lem10.mm.index
2018-05-07 04:39:06,209 : INFO : initializing cython corpus reader from corpus/bow_lem10.mm
2018-05-07 04:39:06,210 : INFO : accepted corpus with 351985 documents, 169716 features, 99651009 non-zero entries


In [9]:
# load p_C
with open('p_C/p_C_lem10.pkl', 'rb') as ip:
    p_C = pickle.load(ip)

In [10]:
# load doc_len
with open('doc_len/doc_len_lem10.pkl', 'rb') as ip:
    doc_len = pickle.load(ip)

In [11]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [12]:

import math
from six import iteritems
from six.moves import xrange

#     The variable k1 is a positive tuning parameter that calibrates the
#     document term frequency scaling. A k1 value of 0 corresponds to a binary
#     model (no term frequency), and a large value corresponds to using raw term
#     frequency. b is another tuning parameter (0 ≤ b ≤ 1) which determines
#     the scaling by document length: b = 1 corresponds to fully scaling the term
#     weight by the document length, while b = 0 corresponds to no length normalization


# BM25 parameters.
PARAM_K1 =1.2
PARAM_K3 =1.2

PARAM_B = 0.75

# incase the idf is zero
EPSILON = 0.01


class BM25(object):

    def __init__(self, corpus):
        self.corpus_size = dictionary.num_docs
        s = 0
        for i in corpus:
            for x in i:
                s = s + x[1]
        s = float(s)    
        self.avgdl = s / self.corpus_size
        self.corpus = corpus
        self.df = dictionary.dfs
        self.idf = {}
        self.initialize()

    def initialize(self):
        for word, freq in iteritems(self.df):
            self.idf[word] = math.log(self.corpus_size) - math.log(freq)

    def get_score(self, query, index, average_idf):
        score = 0
        index_doc = dict(self.corpus[index])
        
        for word,freq in query:
            if word not in index_doc:
                continue
            idf = self.idf[word] if self.idf[word] >= 0 else EPSILON * average_idf
            p1 = ((PARAM_K3 + 1)*freq)/(PARAM_K3 + freq)
            p2 = idf * index_doc[word] * (PARAM_K1 + 1)
            p3 = index_doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len[index] / self.avgdl)
            score += (p1*p2)/p3
        return score

    def get_scores(self, query, average_idf):
        scores = []
        for index in xrange(self.corpus_size):
            score = self.get_score(query, index, average_idf)
            scores.append(score)
        return scores



In [64]:
bm25.idf[2]

1.6774684615627073

In [65]:
bm25.idf[9]

6.190743947288344

In [61]:
dictionary.dfs

{0: 75211,
 1: 107751,
 2: 55413,
 3: 25727,
 4: 12091,
 5: 4134,
 6: 30011,
 7: 39889,
 8: 1643,
 9: 719,
 10: 445,
 11: 9817,
 12: 81120,
 13: 105636,
 14: 90453,
 15: 1628,
 16: 30088,
 17: 72319,
 18: 62311,
 19: 140,
 20: 18203,
 21: 86015,
 22: 43019,
 23: 75202,
 24: 11306,
 25: 3486,
 26: 15168,
 27: 53684,
 28: 27395,
 29: 1188,
 30: 72,
 31: 35670,
 32: 61690,
 33: 3686,
 34: 19359,
 35: 97797,
 36: 19072,
 37: 65850,
 38: 1762,
 39: 929,
 40: 340,
 41: 20352,
 42: 35723,
 43: 52569,
 44: 132,
 45: 84431,
 46: 10903,
 47: 60311,
 48: 25253,
 49: 56046,
 50: 5510,
 51: 94214,
 52: 122,
 53: 41,
 54: 18079,
 55: 14062,
 56: 93829,
 57: 12163,
 58: 8470,
 59: 2962,
 60: 55628,
 61: 5171,
 62: 37000,
 63: 73728,
 64: 86242,
 65: 14680,
 66: 77977,
 67: 57128,
 68: 26819,
 69: 134723,
 70: 31190,
 71: 35037,
 72: 2951,
 73: 80429,
 74: 77159,
 75: 16533,
 76: 56799,
 77: 67923,
 78: 70888,
 79: 8248,
 80: 54834,
 81: 76508,
 82: 44393,
 83: 26563,
 84: 14593,
 85: 7206,
 86: 11194

In [13]:
%%time
bm25 = BM25(corpus)

CPU times: user 1min 10s, sys: 636 ms, total: 1min 11s
Wall time: 1min 11s


In [14]:
%%time
# need to pass it to the function that calculates the score
average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())

CPU times: user 108 ms, sys: 8 ms, total: 116 ms
Wall time: 107 ms


In [ ]:
# x.doesnt_match(words)

# #     Which word from the given list doesn’t go with the others?
# #     Parameters:	words – List of words
# #     Returns:	The word further away from the mean of all words.
# #     Return type:	str

# x.index2word
# ### list of words in the sorted order of their frequency 

# y = x.vocab['parti']
# y.count
# # find freq. of any word


# x.most_similar_cosmul(positive=None, negative=None, topn=10)
# # Find the top-N most similar words, using the multiplicative combination objective proposed by 
# # Omer Levy and Yoav Goldberg.Positive words still contribute positively towards the similarity, negative words 
# # negatively, but with less susceptibility to one large distance dominating the calculation.

# x.most_similar_to_given(entity1, entities_list)
# #    Return the entity from entities_list most similar to entity1.

# x.n_similarity(ws1, ws2)
# #Compute cosine similarity between two sets of words


# x.similar_by_vector(x['fit'], topn=10, restrict_vocab=None)
# # closest vectors
# # if topn is False, similar_by_vector returns the vector of similarity scores.

# x.similar_by_word(word, topn=10, restrict_vocab=None)
# #  Find the top-N most similar words.

# x.similarity(w1, w2)
# #Compute cosine similarity between two words.

# x.wmdistance(doc1, doc2)
# # word movers distance

# y = x.vocab['parti']
# y.count
# #freq in corpus

In [20]:
w2v_model = gensim.models.Word2Vec.load('py_models/W2V_CBOW')

2018-05-07 04:02:30,634 : INFO : loading Word2Vec object from py_models/W2V_CBOW
2018-05-07 04:02:32,933 : INFO : loading vocabulary recursively from py_models/W2V_CBOW.vocabulary.* with mmap=None
2018-05-07 04:02:32,934 : INFO : loading wv recursively from py_models/W2V_CBOW.wv.* with mmap=None
2018-05-07 04:02:32,935 : INFO : loading vectors from py_models/W2V_CBOW.wv.vectors.npy with mmap=None
2018-05-07 04:02:34,280 : INFO : setting ignored attribute vectors_norm to None
2018-05-07 04:02:34,281 : INFO : loading trainables recursively from py_models/W2V_CBOW.trainables.* with mmap=None
2018-05-07 04:02:34,282 : INFO : loading syn1neg from py_models/W2V_CBOW.trainables.syn1neg.npy with mmap=None
2018-05-07 04:02:35,417 : INFO : loading syn1 from py_models/W2V_CBOW.trainables.syn1.npy with mmap=None
2018-05-07 04:02:36,562 : INFO : setting ignored attribute cum_table to None
2018-05-07 04:02:36,564 : INFO : loaded py_models/W2V_CBOW


In [21]:
w2v = w2v_model.wv

In [59]:
c

9

In [60]:
c = 0
for x,y in enumerate(bm25.idf):
    if y < average_idf:
        print dictionary[x]

accord
accordingly
additional
adduce
administration
administrator
admission
adv
affiliate


In [57]:
dictionary[0]

u'accord'

In [56]:
bm25.idf[0]

1.302898950426565

In [51]:
max(bm25.idf.values())

10.41994159272168

In [50]:
c

169716

In [35]:
average_idf

8.950234701004007

In [33]:
bm25.idf[1823]

3.977469283606885

In [31]:
dictionary.token2id['farmer']

1823

In [28]:
w2v.most_similar('farmer')

2018-05-07 04:05:08,514 : INFO : precomputing L2-norms of word weight vectors


[(u'crop', 0.5931399464607239),
 ('agricultural', 0.45630985498428345),
 (u'thefarmers', 0.43082547187805176),
 (u'waiverrelief', 0.4292185306549072),
 ('agriculturist', 0.4271271526813507),
 (u'marginalfarmers', 0.4250454902648926),
 (u'agriculture', 0.4166630804538727),
 ('nodal', 0.41246625781059265),
 ('yield', 0.40330082178115845),
 (u'grow', 0.4020331799983978)]

In [27]:
import numpy as np
np.linalg.norm(w2v['farmer'])

10.7585

In [18]:
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    #query = QE(query)
    queries.append(query)

In [16]:
def QE(query):
    pass
    

SyntaxError: invalid syntax (<ipython-input-16-0f85d86075f9>, line 1)

In [15]:
%%time
results = copy.deepcopy(query_truth)

sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    query = QE(query)
    queries.append(query)
    sims = bm25.get_scores(dictionary.doc2bow(query), average_idf)
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
    print i
   
        

0
1
2
3
4
5
6
7
8
9
CPU times: user 8min 19s, sys: 16.6 s, total: 8min 36s
Wall time: 8min 36s


In [16]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 56.29873733052213, 437],
  ['ConsumerCourt_DCDRC_106530', '0', 53.842739768299, 649],
  ['ConsumerCourt_DCDRC_107608', '1', 55.18797260487204, 529],
  ['ConsumerCourt_DCDRC_114291', '1', 71.08979242641004, 55],
  ['ConsumerCourt_DCDRC_114382', '1', 71.89079068217174, 48],
  ['ConsumerCourt_DCDRC_118185', '1', 76.15605830990927, 20],
  ['ConsumerCourt_DCDRC_130318', '0', 67.62877028242994, 85],
  ['ConsumerCourt_DCDRC_130570', '1', 75.18840269019088, 23],
  ['ConsumerCourt_DCDRC_131146', '1', 77.71060812434385, 13],
  ['ConsumerCourt_DCDRC_131717', '1', 65.53276847507685, 117],
  ['ConsumerCourt_DCDRC_131741', '1', 76.15931252726007, 19],
  ['ConsumerCourt_DCDRC_131818', '1', 80.61692224075088, 6],
  ['ConsumerCourt_DCDRC_131950', '0', 69.3323776680298, 69],
  ['ConsumerCourt_DCDRC_131972', '0', 98.0375316619314, 1],
  ['ConsumerCourt_DCDRC_132932', '0', 72.53483993142522, 42],
  ['ConsumerCourt_DCDRC_133592', '1', 76.14355395485467, 21],
  ['C

In [17]:
model_name = 'BM25_lem10_Ex'

with open('py_results/' + model_name + 'sim_list.pkl', 'wb') as output:
    pickle.dump(sim_list, output, pickle.HIGHEST_PROTOCOL)
with open('py_results/' + model_name + 'rank_list.pkl', 'wb') as output:
    pickle.dump(rank_list, output, pickle.HIGHEST_PROTOCOL)


In [18]:
# Store results in 1st sheet


wb = Workbook()
dest_filename = 'result_excel/' + model_name + '.xlsx'
ws1 = wb.active
ws1.title = "ground_truth"
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)

#Query wise (F score)

import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean



places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name ]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name ]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = dest_filename)


#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean


places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):
        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name ]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name ]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = dest_filename)

In [4]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, index= False, header = False, **to_excel_kwargs)

    # save the workbook
    writer.save()

model_name = 'BM25_lem10_Ex'
dest_filename = 'result_excel/' + model_name + '.xlsx'
    
f = pd.ExcelFile(dest_filename)
sec_dest = 'result_excel/all_results.xlsx'
f2 = pd.ExcelFile(sec_dest)
append_df_to_excel(sec_dest, f.parse(1), sheet_name= f2.sheet_names[0])
append_df_to_excel(sec_dest, f.parse(2), sheet_name= f2.sheet_names[1])

In [ ]:
###############################################################################

In [20]:
def Top_x(x, R):
    R = list(R)
    x_names = []
    for i in range(x):
        x_names.append(filenames[R.index(i + 1)].strip('.txt'))
    return x_names
    
    

In [26]:
from shutil import copy2

In [54]:
for i in range(1, 11):
    destdir = 'mini_corpus/q'+str(i)
    os.makedirs(destdir)



In [55]:
for i in range(1, 10):
    destdir = 'mini_corpus/q'+str(i) 
    copy2('LegalAdhocTask/q' + str(i) +'.txt', destdir)


destdir = 'mini_corpus/q'+str(10) +'/q10.txt' 
copy2('LegalAdhocTask/q' + str(91) +'.txt', destdir) 

In [44]:
 Top_x(15, rank_list[i])

['ConsumerCourt_DCDRC_119856',
 'ConsumerCourt_DCDRC_227115',
 'ConsumerCourt_DCDRC_222047',
 'ConsumerCourt_DCDRC_133010',
 'ConsumerCourt_DCDRC_131385',
 'ConsumerCourt_DCDRC_143537',
 'ConsumerCourt_DCDRC_224718',
 'ConsumerCourt_DCDRC_126643',
 'ConsumerCourt_DCDRC_56957',
 'ConsumerCourt_DCDRC_108097',
 'ConsumerCourt_DCDRC_227084',
 'ConsumerCourt_DCDRC_45418',
 'ConsumerCourt_SCDRC_59307',
 'ConsumerCourt_DCDRC_131387',
 'ConsumerCourt_SCDRC_69433']

In [49]:
docs = query_truth[str(i)]
docs.sort(key = lambda x : -int(x[1]))
docs = [ j[0] for j in docs if j[1] == str(1)] 
docs2 = Top_x(15, rank_list[i])
docs =  set.intersection(set(docs), set(docs2))

In [52]:
x

{'ConsumerCourt_DCDRC_131146',
 'ConsumerCourt_DCDRC_131818',
 'ConsumerCourt_DCDRC_217344',
 'ConsumerCourt_DCDRC_222844',
 'ConsumerCourt_DCDRC_39186',
 'ConsumerCourt_DCDRC_41317',
 'ConsumerCourt_DCDRC_41588',
 'ConsumerCourt_DCDRC_42649',
 'ConsumerCourt_DCDRC_46519',
 'ConsumerCourt_DCDRC_53138',
 'ConsumerCourt_DCDRC_55678',
 'ConsumerCourt_DCDRC_55899'}

In [51]:
docs2

['ConsumerCourt_DCDRC_131972',
 'ConsumerCourt_DCDRC_139024',
 'ConsumerCourt_DCDRC_53138',
 'ConsumerCourt_DCDRC_41588',
 'ConsumerCourt_DCDRC_55899',
 'ConsumerCourt_DCDRC_131818',
 'ConsumerCourt_DCDRC_217344',
 'ConsumerCourt_DCDRC_41317',
 'ConsumerCourt_DCDRC_55678',
 'ConsumerCourt_DCDRC_39186',
 'ConsumerCourt_DCDRC_46519',
 'ConsumerCourt_DCDRC_222695',
 'ConsumerCourt_DCDRC_131146',
 'ConsumerCourt_DCDRC_42649',
 'ConsumerCourt_DCDRC_222844']

In [50]:
docs


['ConsumerCourt_DCDRC_100385',
 'ConsumerCourt_DCDRC_107608',
 'ConsumerCourt_DCDRC_114291',
 'ConsumerCourt_DCDRC_114382',
 'ConsumerCourt_DCDRC_118185',
 'ConsumerCourt_DCDRC_130570',
 'ConsumerCourt_DCDRC_131146',
 'ConsumerCourt_DCDRC_131717',
 'ConsumerCourt_DCDRC_131741',
 'ConsumerCourt_DCDRC_131818',
 'ConsumerCourt_DCDRC_133592',
 'ConsumerCourt_DCDRC_135474',
 'ConsumerCourt_DCDRC_140039',
 'ConsumerCourt_DCDRC_187214',
 'ConsumerCourt_DCDRC_207784',
 'ConsumerCourt_DCDRC_217344',
 'ConsumerCourt_DCDRC_217471',
 'ConsumerCourt_DCDRC_222844',
 'ConsumerCourt_DCDRC_226072',
 'ConsumerCourt_DCDRC_39186',
 'ConsumerCourt_DCDRC_41317',
 'ConsumerCourt_DCDRC_41588',
 'ConsumerCourt_DCDRC_42118',
 'ConsumerCourt_DCDRC_42618',
 'ConsumerCourt_DCDRC_42649',
 'ConsumerCourt_DCDRC_44200',
 'ConsumerCourt_DCDRC_46368',
 'ConsumerCourt_DCDRC_46519',
 'ConsumerCourt_DCDRC_46543',
 'ConsumerCourt_DCDRC_53138',
 'ConsumerCourt_DCDRC_55517',
 'ConsumerCourt_DCDRC_55612',
 'ConsumerCourt_DCDRC

In [57]:
for i in range(1, 11):
    docs = query_truth[str(i)]
    docs.sort(key = lambda x : -int(x[1]))
    docs = [ j[0] for j in docs if j[1] == str(1)] 
    docs2 = Top_x(15, rank_list[i])
    docs =  set(docs2) - set(docs)
    destdir = 'mini_corpus/q'+str(i)
    for doc in docs:
        src = full_filenames[filenames.index(doc + '.txt')] 
        #print src
        copy2(src, destdir)
    #os.makedirs(destdir)
    print i

1
2
3
4
5
6
7
8
9
10


In [ ]:
############################################################

In [ ]:
def Top_x(x, R):
    R = list(R)
    x_names = []
    for i in range(x):
        x_names.append(filenames[R.index(i + 1)].strip('.txt'))
    return x_names
    
    

In [ ]:
def score_stat(word, freq, index):
    index_doc = dict(bm25.corpus[index])
    word = dictionary.doc2bow([word])[0][0]
    idf = bm25.idf[word] if bm25.idf[word] >= 0 else EPSILON * average_idf
    score = freq*((idf * index_doc[word] * (PARAM_K1 + 1)
              / (index_doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len[index] / bm25.avgdl))))
    return (index_doc[word], score)

In [ ]:
wb = Workbook()
dest_filename = 'Temp.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)
def common_words(q, F):
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    index = filenames.index(F + '.txt')
    index_doc = dict(bm25.corpus[index])
    row = []
    for word, freq in  zip(q_words, q_freq):
        w = dictionary.doc2bow([word])[0][0]
        if w in index_doc: 
            f, s = score_stat(word, freq, index)
            row.append([word + ', ' + str(round(bm25.idf[w], 2)), freq, f,  round( (s*100)/sim_list[q][index], 1)])  
    row.sort(key = lambda x: (x[3],x[1],x[2]), reverse = True)
    wb = load_workbook('Temp.xlsx')
    ws = wb.create_sheet(title=str(q) + F)
    ws.append(['word,idf', 'Query freq', 'freq', '% score'])
    for i in row:
        ws.append(i)
    wb.save(filename = 'Temp.xlsx')


In [ ]:
f = open("toberead.txt", 'r') 
count = 0
q = 1
for i in f:
    if i == '\n':
        continue
    count = count + 1
    common_words(q, i.strip())
    if count % 5 == 0:
        q = q + 1


In [ ]:
for i in files_to_tokens(full_filenames[filenames.index(doc + '.txt')]):
    for j in i:
        print j

In [ ]:
doc = 'ConsumerCourt_SCDRC_30176'
corpus[]

In [ ]:
common_words(1, 'ConsumerCourt_DCDRC_41588')

In [ ]:
df = pd.DataFrame(index= None, columns=['words','pair'])


In [ ]:
df

In [ ]:
print df

In [ ]:
df.append(['lol', ('we','wew')])

In [ ]:
def uncommon_words(q, filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)

wb = load_workbook('Explore_BM25.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_uncom.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_uncom.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
#         for word, freq in  zip(q_words, q_freq):
#             if dictionary.doc2bow([word])[0][0] in index_doc: 
#                 f, s = score_stat(word, freq, index)
#                 row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
#             else:
#                 row.append('X')
        #ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_all.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_all.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(sec_row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)